# Analysis report for GEO1001 HW01 
---
  
## Information of authors  
1. First author: 
 
   * [<img src="https://avatars3.githubusercontent.com/u/59593272?s=400&u=ba1618be6d5e354f0bd7685ff405bdec6d18c101&v=4" width = "50" height = "50" />](https://github.com/peterliu502)  
   * Name: Zhenyu Liu  
   * Student Number: 5386586  

2. Second author:
  
  * [<img src="https://avatars2.githubusercontent.com/u/47234206?s=400&u=3f54e18f68e48f985db9f0ef1a8eb3a3ca189b1e&v=4" width = "50" height = "50" />](https://github.com/Ziyan-Wu)
  * Name: Ziyan Wu
  * Student Number: 5360684
  
## Source data: